# Importing ARCOS Data with Dask

Last week, we used dask to play with a few datasets to get a feel for how dask works. In order to help us develop code that would run quickly, however, we worked with very small, safe datasets. 

Today, we will continue to work with dask, but this time using much larger datasets. This means that (a) doing things incorrectly may lead to your computer crashing (So save all your open files before you start!), and (b) many of the commands you are being asked run will take several minutes each. 

For familiarity, and so you can see what advantages dask can bring to your workflow, today we'll be working with the DEA ARCOS drug shipment database published by the Washington Post! However, to strike a balance between size and speed, we'll be working with a slightly thinned version that has only the last two years of data, instead of all six.

## Exercise 1

Download the thinned ARCOS data [from this link](https://www.dropbox.com/s/o7nc6yvrwog4ozi/arcos_2011_2012.tsv.zip?dl=0). It should be about 2GB zipped, 25 GB unzipped. 

In [1]:
# importing libraries
import numpy as np
import pandas as pd

pd.set_option("mode.copy_on_write", True)

## Exercise 2

Our goal today is going to be to find the pharmaceutical company that has shipped the most opioids (`MME_Conversion_Factor * CALC_BASE_WT_IN_GM`) in the US.

When working with large datasets, it is good practice to begin by prototyping your code with a subset of your data. So begin by using `pandas` to read in the first 100,000 lines of the ARCOS data and write pandas code to compute the shipments from each shipper (the group that reported the shipment). 

In [2]:
# reading initial data for analysis
df = pd.read_csv("/Users/revanth/Downloads/arcos_2011_2012.tsv", sep="\t", nrows=100000)
df.head()

/var/folders/cj/jkp7xyld4_q3x3rk7xdx7jwh0000gn/T/ipykernel_47889/2822659403.py:2: DtypeWarning: Columns (4,6,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/revanth/Downloads/arcos_2011_2012.tsv", sep="\t", nrows=100000)


,Unnamed: 0,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,...,Product_Name,Ingredient_Name,Measure,MME_Conversion_Factor,Combined_Labeler_Name,Revised_Company_Name,Reporter_family,dos_str,date,year
0,0,PA0006836,DISTRIBUTOR,ACE SURGICAL SUPPLY CO INC,NaN,1034 PEARL STREET,NaN,BROCKTON,MA,2301,...,HYDROCODONE BIT/ACETA 10MG/500MG USP,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,SpecGx LLC,Mallinckrodt,ACE Surgical Supply Co Inc,10.0,2012-12-26,2012
1,9,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,...,HYDROCODONE BITARTRATE & ACETA 5MG/,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Apotheca Inc.,Apotheca Inc.,Apotheca Inc,5.0,2012-12-05,2012
2,10,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,...,HYDROCODONE BITARTRATE & ACETA 5MG/,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Apotheca Inc.,Apotheca Inc.,Apotheca Inc,5.0,2012-07-24,2012
3,16,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,...,HYDROCODONEBITARTRATE & ACETA 7.5MG,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Apotheca Inc.,Apotheca Inc.,Apotheca Inc,7.5,2012-02-04,2012
4,17,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,...,HYDROCODONE BITARTRATE & ACETA 5MG/,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Apotheca Inc.,Apotheca Inc.,Apotheca Inc,5.0,2011-11-07,2011


In [3]:
# calculating shipement in MME
df["MME"] = df["MME_Conversion_Factor"] * df["CALC_BASE_WT_IN_GM"]

In [4]:
# calculating Shipment by Reporter
df2 = df.groupby("REPORTER_NAME")["MME"].sum().reset_index()
df2.head()

,REPORTER_NAME,MME
0,ACE SURGICAL SUPPLY CO INC,0.605400
1,AMERICAN SALES COMPANY,3432.058005
2,AMERISOURCEBERGEN DRUG CORP,34561.394892
3,APOTHECA INC,23.913300
4,BLOODWORTH WHOLESALE DRUGS,1782.827325


## Exercise 3

Now let's turn to dask. Re-write your code for dask, and calculate the total shipments by reporting company. Remember: 

- Activate a conda environment with a clean dask installation.
- Start by spinning up a distributed cluster.
- Dask won't read compressed files, so you have to unzip your ARCOS data. 
- Start your cluster in a cell all by itself since you don't want to keep re-running the "start a cluster" code. 

If you need to review dask basic code, [check here](https://nickeubank.github.io/practicaldatascience_book/notebooks/PDS_not_yet_in_coursera/30_big_data/70_dask.html).

As you run your code, make sure to click on the Dashboard link below where you created your cluster:

![dask_dashboard](images/dask_cluster.png)

Among other things, the bar across the bottom should give you a sense of how long your task will take:

![dask_progress](images/dask_progress.png)

(For context, my computer (which has 10 cores) only took a couple seconds. My computer is fast, but most computers should be done within a couple minutes, tops).


In [5]:
# Importig Dask and creating a client
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64668,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:64679,Total threads: 2
Dashboard: http://127.0.0.1:64680/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:64671,


In [6]:
import dask.dataframe as dd

In [7]:
# reading data using dask
dask_df = dd.read_csv(
    "/Users/revanth/Downloads/arcos_2011_2012.tsv",
    sep="\t",
    dtype={
        "ORDER_FORM_NO": "object",
        "REPORTER_ADDL_CO_INFO": "object",
        "REPORTER_ADDRESS2": "object",
        "ACTION_INDICATOR": "object",
        "NDC_NO": "object",
        "UNIT": "object",
    },
    assume_missing=True,
)

In [8]:
# Performing same operations as performed on pandas dataframe
dask_df["MME"] = dask_df["MME_Conversion_Factor"] * dask_df["CALC_BASE_WT_IN_GM"]
dask_df_2 = dask_df.groupby("REPORTER_NAME")["MME"].sum().reset_index()
output_1 = dask_df_2.compute()

/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataf

In [9]:
# Viewing the output
output_1.head()

,REPORTER_NAME,MME
0,ACE SURGICAL SUPPLY CO INC,2.421600e+00
1,AMERICAN SALES COMPANY,2.798722e+04
2,AMERISOURCEBERGEN DRUG CORP,3.999211e+06
3,APOTHECA INC,1.935657e+02
4,BLOODWORTH WHOLESALE DRUGS,1.048958e+04


## Exercise 4

Now let's calculate, *for each state*, what company shipped the most pills?

Note you will quickly find that you can't sort in dask -- sorting in parallel is *really* tricky! So you'll have to work around that. Do what you need to do on the big dataset first, then compute it all so you get it as a regular pandas dataframe, then finish. 

Does this seem like a situation where a single company is responsible for the opioid epidemic?

In [10]:
# Calcualting shipment by state and reporter
dask_df_3 = dask_df.groupby(["BUYER_STATE", "REPORTER_NAME"])["MME"].sum().reset_index()

# getting max shipment by state
dask_df_4 = dask_df_3.groupby("BUYER_STATE")["MME"].max().reset_index()

# merging the dataframes to get reporter with max shipment by state
dask_df_5 = dask_df_4.merge(dask_df_3, on=["BUYER_STATE", "MME"], how="left")
output_2 = dask_df_5.compute()

/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataf

In [11]:
output_2.head()

,BUYER_STATE,MME,REPORTER_NAME
0,AK,19447.768327,CARDINAL HEALTH
1,AL,313102.490038,MCKESSON CORPORATION
2,AR,97547.962187,MCKESSON CORPORATION
3,AZ,343528.893987,MCKESSON CORPORATION
4,CA,801977.350276,MCKESSON CORPORATION


In [12]:
# getting number of states each reporter is the max shipment reporter
output_2["REPORTER_NAME"].value_counts()

REPORTER_NAME
MCKESSON CORPORATION              24
CARDINAL HEALTH                   14
WALGREEN CO                        6
AMERISOURCEBERGEN DRUG CORP        4
AMERISOURCEBERGEN DRUG             1
LOUISIANA WHOLESALE DRUG CO        1
MCKESSON DRUG COMPANY              1
DAKOTA DRUG                        1
CARDINAL HEALTH 110, LLC           1
DROGUERIA BETANCES, LLC            1
CARDINAL HEALTH P.R. 120, INC.     1
Name: count, dtype: int64[pyarrow]

# Observations
Majority of the opioid shipments seem to be from 2 companies: MCKESSON CORPORATION and CARDINAL HEALTH

## Exercise 5 

Now go ahead and try and re-do the chunking you did by hand for your project (with this 2 years of data) -- calculate, for each year, the total morphine equivalents sent to each county in the US. 

In [13]:
# Calculating shipment by state, county and year
dask_df_6 = (
    dask_df.groupby(["BUYER_STATE", "BUYER_COUNTY", "year"])["MME"].sum().reset_index()
)
output_3 = dask_df_6.compute()

/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/revanth/miniconda3/envs/dask_env1/lib/python3.11/site-packages/dask/dataf

In [14]:
# viewing the output
output_3.head()

,BUYER_STATE,BUYER_COUNTY,year,MME
0,AL,AUTAUGA,2011.0,4181.623992
1,AL,AUTAUGA,2012.0,4346.607187
2,AL,BARBOUR,2011.0,1317.385035
3,AL,BARBOUR,2012.0,1385.788223
4,AL,BUTLER,2011.0,1186.474868


## Exercise 6

Now, re-write your opioid project's initial opioid import using dask. Each person on your team should create a NEW branch to try this. The person who wrote the initial chunking code can help everyone else understand what they did originally and the data, but everyone should write their own code. 

**WARNING:** You will probably run into a lot of type errors (depending on how the ARCOS data has changed since last year). With real world messy data one of the biggest problems with dask is that it struggles if halfway through dataset it discovers that the column it *thought* was floats contains text. That's why, in the dask reading, [I specified the column type for so many columns](https://nickeubank.github.io/practicaldatascience_book/notebooks/PDS_not_yet_in_coursera/30_big_data/70_dask.html#what-can-dask-do-for-me) as `objects` explicitly. Then, because occasionally there data cleanliness issues, I had to do some converting data types by hand. 